In [1]:
!pip install allennlp-models
!pip install urllib3
!pip install awscli --ignore-installed six
#BeautifulSoup and requests
!pip install beautifulsoup4
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 464 kB 28.9 MB/s 
     |████████████████████████████████| 56 kB 1.1 MB/s 
     |████████████████████████████████| 730 kB 74.9 MB/s 
     |████████████████████████████████| 53 kB 984 kB/s 
     |████████████████████████████████| 451 kB 91.0 MB/s 
     |████████████████████████████████| 125 kB 89.5 MB/s 
     |████████████████████████████████| 305 kB 93.8 MB/s 
     |████████████████████████████████| 248 kB 95.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 4.5 MB 78.7 MB/s 
     |████████████████████████████████| 107 kB 85.0 MB/s 
     |████████████████████████████████| 593 kB 68.9 MB/s 
     |████████████████████████████████| 316 kB 72.4 MB/s 
     |████████████████████████████████| 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


<!-- {cancer:
{
  food:fish 
  result: positve}
  } -->

In [2]:
from allennlp_models.pretrained import load_predictor
import numpy as np
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
## Function for deciding if a food is beneficial or detrimental

# Uses the result of the prob_for_single_food/prob_for_single_food_single_line dict in order to decide
# If max(pos_entailment_prob) > max(pos_neutral_prob) and max(neg_entailment_prob) -> Sort food as "positive"
# If max(neg_entailment_prob) > max(neg_neutral_prob) and max(pos_entailment_prob) -> Sort food as "negative"
# Else -> Sort food as "neutral"

# if at some point check each line
# // pos entailment > pos neutral and pos entailment>0.8 => pos
# neg entailment > neg neutral and neg entailment>0.8 => neg
# pos contradiction > pos neutral and pos contra>0.8 = > neg
# neg contradiction > neg neutral and neg contra>0.8 = > pos

def sort_food(prob_dict, threshold = 0.8): # Takes in a dictionary of probabilities, along with a threshold value, returns a String result and the probability
  # pos_prob = max(prob_dict['pos']['entailment'])
  # pos_neutral_prob = max(prob_dict['pos']['neutral'])
  # neg_prob = max(prob_dict['neg']['entailment'])
  # neg_neutral_prob = max(prob_dict['neg']['neutral'])
  pos_entailment = prob_dict['pos']['entailment']
  pos_neutral = prob_dict['pos']['neutral']
  pos_contradiction = prob_dict['pos']['contradiction']

  neg_entailment = prob_dict['neg']['entailment']
  neg_neutral = prob_dict['neg']['neutral']
  neg_contradiction = prob_dict['neg']['contradiction']

  for i in range(len(pos_entailment)): # We need separate loops since the number of sentences used for positive sentiment may be different than negative
    if pos_entailment[i] > pos_neutral[i] and pos_entailment[i] >= threshold:
      return "Positive", pos_entailment[i]
    elif pos_contradiction[i] > pos_neutral[i] and pos_contradiction[i] >= threshold:
      return "Negative", pos_contradiction[i]
  
  for i in range(len(neg_entailment)):
    if neg_entailment[i] > neg_neutral[i] and neg_entailment[i] >= threshold:
      return "Negative", neg_entailment[i]
    elif neg_contradiction[i] > neg_neutral[i] and neg_contradiction[i] >= threshold:
      return "Positive", neg_contradiction[i]

  return "Neutral", max(pos_neutral)

  # if (pos_prob > pos_neutral_prob and pos_prob > neg_prob):
  #   return "Positive", pos_prob # Should something else be done here as a double check?
  # elif (neg_prob > neg_neutral_prob and neg_prob > pos_prob):
  #   return "Negative", neg_prob # Should something else be done here as a double check?
  # else:
  #   return "Neutral", pos_neutral_prob # If those conditions aren't met, then we can say that the outcome is inconclusive and therefore neutral


In [5]:
# food_name - str, name of a single food ('ground chicken')
# diet_reco_lines - iterable (list) [each item will be a str, a single sentence froma  Wiki paragraph]

def prob_for_single_food(food_name, diet_reco_lines): # a single food-disease pair
    predictor = load_predictor("pair-classification-roberta-snli")
    print('Diet recommendation has', len(diet_reco_lines), 'lines')

    # Positive association check
    pos_entailment_prob = [] # entailment_prob for a pos association of food_name with each line in diet_reco_lines
    pos_contradiction_prob = [] # contradiction_prob for a pos association of food_name with each line in diet_reco_lines
    pos_neutral_prob = [] # neutral_prob for a pos association of food_name with each line in diet_reco_lines
    # pos_phrases = ['should be chosen']
    pos_phrases = ['is good', 'is recommended', 'is beneficial', 'is healthy', 'is useful', 'is highly recommended', 'is highly beneficial', 'is very healthy', 'is very good', 'is very useful','should be chosen']
    for line in diet_reco_lines:
        premise = line
        ep = [] # 10 entailment prob
        cp = []
        np = []
        for phrase in pos_phrases:
            hypothesis = food_name + ' ' + phrase
            probs = predictor.predict(premise, hypothesis)["probs"]
            ep.append(probs[0])
            cp.append(probs[1])
            np.append(probs[2])
        pos_entailment_prob.append(max(ep))
        pos_contradiction_prob.append(max(cp))
        pos_neutral_prob.append(max(np))

    # Negative association check
    neg_entailment_prob = [] # entailment_prob for a neg association of food_name with each line in diet_reco_lines
    neg_contradiction_prob = [] # contradiction_prob for a neg association of food_name with each line in diet_reco_lines
    neg_neutral_prob = [] # neutral_prob for a neg association of food_name with each line in diet_reco_lines
    # neg_phrases = ['should not be chosen']
    neg_phrases = ['is risky', 'is harmful', 'is very harmful', 'is discouraged', 'is highly discouraged', 'is very risky', 'is unhealthy', 'is very unhealthy', 'is bad', 'is very bad','should not be chosen']
    for line in diet_reco_lines:
        premise = line
        ep = []
        cp = []
        np = []
        for phrase in neg_phrases:
            hypothesis = food_name + ' ' + phrase
            probs = predictor.predict(premise, hypothesis)["probs"]
            ep.append(probs[0])
            cp.append(probs[1])
            np.append(probs[2])
        neg_entailment_prob.append(max(ep))
        neg_contradiction_prob.append(max(cp))
        neg_neutral_prob.append(max(np))

    prob_dict = {
        'pos': {
            'entailment': pos_entailment_prob,
            'contradiction': pos_contradiction_prob,
            'neutral': pos_neutral_prob
        },
        'neg': {
            'entailment': neg_entailment_prob,
            'contradiction': neg_contradiction_prob,
            'neutral': neg_neutral_prob
        }
    }

    # prob_arr = np.array([
    #     [pos_entailment_prob, pos_contradiction_prob, pos_neutral_prob],
    #     [neg_entailment_prob, neg_contradiction_prob, neg_neutral_prob]
    # ])

    #return prob_arr
    print(prob_dict)
    return prob_dict

In [6]:
file = open("/content/drive/MyDrive/diseases_diet.json", 'r')

json_data = json.load(file)

# print(json_data["Diseases"])

#lis = []

# for disease in json_data:
#   # print(disease)
#   for line in json_data[disease]:

#     print(i)
  
file.close()

In [7]:
path = '/content/drive/MyDrive/food_subgroup.csv' ##Use the link to the file link in google drive//Drive mount in local drive

import pandas as pd
import io
df = pd.read_csv(path)
#print(df)

df2 = df[['food_subgroup']]
# print(df2)

col_list = df2['food_subgroup'].values.tolist() #col_list can be iterated to get food item.
#print(col_list)

In [8]:
from tables.tests.common import ne
#List - all the lines:
sentiment_list =[]
neutral_list =[]

for disease in json_data:
  # print(disease)
  for line in json_data[disease]:
    for food in col_list:
      if food.lower() in line.lower():
        sentiment_list.append(food)
      else:
         neutral_list.append(food)


neutral_list = [*set(neutral_list)]
sentiment_list = [*set(sentiment_list)]

dict_lines = {}

for food in sentiment_list:
  for disease in json_data:
    for line in json_data[disease]:
      if food in line:
        dict_lines[disease] = line

print(dict_lines)


# print(str(len(neutral_list)))
for x in sentiment_list:
  print(str(x))


{'Stroke': 'Nutrition, specifically the Mediterranean-style diet, has the potential for decreasing the risk of having a stroke by more than half. It does not appear that lowering levels of homocysteine with folic acid affects the risk of stroke.', 'Dementia': 'The Mediterranean and DASH diets are both associated with less cognitive decline. A different approach has been to incorporate elements of both of these diets into one known as the MIND diet. These diets are generally low in saturated fats while providing a good source of carbohydrates, mainly those that help stabilize blood sugar and insulin levels. Raised blood sugar levels over a long time, can damage nerves and cause memory problems if they are not managed. Nutritional factors associated with the proposed diets for reducing dementia risk include unsaturated fatty acids, vitamin E, vitamin C, flavonoids, vitamin B, and vitamin D.', 'Gallstone': 'Nutritional factors that may increase risk of gallstones include constipation; eat

In [ ]:
output_dict ={}
for disease in dict_lines:
  print(dict_lines[disease])
  output_dict[disease] = {}
  for food in sentiment_list:
      dict_tool = prob_for_single_food(food,[dict_lines[disease]])
      txt, prob_num =  sort_food(dict_tool)
      # output_dict[disease]["food"] = food
      output_dict[disease][food] = txt
      print(food+" "+" Txt: "+txt+"Prob: "+str(prob_num))

second_dict = output_dict

Nutrition, specifically the Mediterranean-style diet, has the potential for decreasing the risk of having a stroke by more than half. It does not appear that lowering levels of homocysteine with folic acid affects the risk of stroke.


Diet recommendation has 1 lines
{'pos': {'entailment': [0.0025365285109728575], 'contradiction': [0.9958754181861877], 'neutral': [0.019484316930174828]}, 'neg': {'entailment': [0.0013569140573963523], 'contradiction': [0.9969733953475952], 'neutral': [0.003533224808052182]}}
Vegetable  Txt: NegativeProb: 0.9958754181861877


Diet recommendation has 1 lines
{'pos': {'entailment': [0.08307702839374542], 'contradiction': [0.9903610944747925], 'neutral': [0.09252802282571793]}, 'neg': {'entailment': [0.0014678792795166373], 'contradiction': [0.9966975450515747], 'neutral': [0.003032419364899397]}}
Mediterranean Diet  Txt: NegativeProb: 0.9903610944747925


Diet recommendation has 1 lines
{'pos': {'entailment': [0.0016466378001496196], 'contradiction': [0.9950633645057678], 'neutral': [0.020404180511832237]}, 'neg': {'entailment': [0.001560427132062614], 'contradiction': [0.9965772032737732], 'neutral': [0.004900775384157896]}}
Fruit  Txt: NegativeProb: 0.9950633645057678


Diet recommendation has 1 lines
{'pos': {'entailment': [0.0012345786672085524], 'contradiction': [0.9970843195915222], 'neutral': [0.0060095167718827724]}, 'neg': {'entailment': [0.0013989864382892847], 'contradiction': [0.9972758889198303], 'neutral': [0.0026431235019117594]}}
Seaweed  Txt: NegativeProb: 0.9970843195915222


...

Diet recommendation has 1 lines
{'pos': {'entailment': [0.032096996903419495], 'contradiction': [0.5337309241294861], 'neutral': [0.8590999245643616]}, 'neg': {'entailment': [0.036942530423402786], 'contradiction': [0.5304931998252869], 'neutral': [0.9262622594833374]}}
Fat  Txt: NeutralProb: 0.8590999245643616


Diet recommendation has 1 lines
{'pos': {'entailment': [0.0869516059756279], 'contradiction': [0.4777578115463257], 'neutral': [0.9357423186302185]}, 'neg': {'entailment': [0.020304854959249496], 'contradiction': [0.7952901721000671], 'neutral': [0.8235703706741333]}}
Beverages  Txt: NeutralProb: 0.9357423186302185


Diet recommendation has 1 lines
{'pos': {'entailment': [0.0036994945257902145], 'contradiction': [0.8517248034477234], 'neutral': [0.4526064693927765]}, 'neg': {'entailment': [0.0018231479916721582], 'contradiction': [0.9420689940452576], 'neutral': [0.36969491839408875]}}
Meat  Txt: NegativeProb: 0.8517248034477234


Diet recommendation has 1 lines
{'pos': {'entailment': [0.02145012468099594], 'contradiction': [0.5456798076629639], 'neutral': [0.8101754784584045]}, 'neg': {'entailment': [0.010167310945689678], 'contradiction': [0.862823486328125], 'neutral': [0.5442849397659302]}}
Legumes  Txt: PositiveProb: 0.862823486328125


Diet recommendation has 1 lines
{'pos': {'entailment': [0.013816270045936108], 'contradiction': [0.5759490132331848], 'neutral': [0.8443469405174255]}, 'neg': {'entailment': [0.005560094490647316], 'contradiction': [0.6099353432655334], 'neutral': [0.7714299559593201]}}
Animal fat  Txt: NeutralProb: 0.8443469405174255


In [1]:
print(output_dict)

for disease in output_dict:
  for food in neutral_list:
    # output_dict[disease]["food"] = food
    output_dict[disease][food] = "neutral"

#converting dict to json
with open("diet_output.json", "w") as outfile:
    json.dump(output_dict, outfile)

NameError: ignored